In [1]:
!nvidia-smi

Mon Dec  4 16:20:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers

In [3]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#data reading
patents=pd.read_csv("/content/drive/MyDrive/BERT/patents_data000000000000.csv")
# df1=pd.read_csv("/content/drive/MyDrive/patents_data000000000001.csv",encoding="unicode_escape",on_bad_lines="skip")
# df2=pd.read_csv("/content/drive/MyDrive/patents_data000000000002.csv",encoding="unicode_escape",on_bad_lines="skip")
# df3=pd.read_csv("/content/drive/MyDrive/patents_data000000000003.csv",encoding="unicode_escape",on_bad_lines="skip")
# df4=pd.read_csv("/content/drive/MyDrive/patents_data000000000004.csv",encoding="unicode_escape",on_bad_lines="skip")

In [6]:
# patents=pd.concat([df0,df1,df2,df3,df4])
# patents
# patents.shape

In [7]:
patents=patents.drop(["id","date"],axis=1)
patents

,cpc_ids,text
0,"B01D,C01B,C23F,C25D,H01M,Y02E,Y10T",1. A production method of a hydrogen productio...
1,G11C,1. A method comprising: storing states in one ...
2,"B65H,G03G","1. An image-forming apparatus, comprising: a b..."
3,"H01H,H01R",1. A system for coupling with a first eService...
4,"G06F,H04L","1. A system, for generating a web page portal ..."
...,...,...
55481,H04B,1. A first node in a wireless communication sy...
55482,G11C,"1. An internal voltage generator circuit, comp..."
55483,G01C,1. A route searching apparatus capable of usin...
55484,C21D,1. A quench ring for ejecting quenchant onto a...


In [8]:
import pandas as pd
#Code to create dummy features regarding the type of cuisines in the dataset.
patents_data = pd.concat([patents.drop('cpc_ids', axis=1), patents['cpc_ids'].str.get_dummies(sep=",")], axis=1)
patents_data

,text,A01B,A01C,A01D,A01F,A01G,A01H,A01J,A01K,A01M,...,Y02B,Y02C,Y02D,Y02E,Y02P,Y02T,Y02W,Y04S,Y10S,Y10T
0,1. A production method of a hydrogen productio...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,1. A method comprising: storing states in one ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"1. An image-forming apparatus, comprising: a b...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1. A system for coupling with a first eService...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"1. A system, for generating a web page portal ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55481,1. A first node in a wireless communication sy...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55482,"1. An internal voltage generator circuit, comp...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55483,1. A route searching apparatus capable of usin...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55484,1. A quench ring for ejecting quenchant onto a...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#This cell needs to run when we combined all the dataframes, it's not needed for only 1st df
"""
patents1 = patents_data.drop(patents_data.columns[1], axis=1)
patents1.head()
patents1.shape
"""

'\npatents1 = patents_data.drop(patents_data.columns[1], axis=1)\npatents1.head()\npatents1.shape\n'

In [10]:
"""
This implementation is not needed for 1st df
patents1 = patents1.drop(patents1.columns[1], axis=1)
patents1.head()
"""

'\nThis implementation is not needed for 1st df \npatents1 = patents1.drop(patents1.columns[1], axis=1)\npatents1.head()\n'

In [11]:
patents1=patents_data[:100]

In [12]:
target_list=patents1.columns[1:]
target_list

Index(['A01B', 'A01C', 'A01D', 'A01F', 'A01G', 'A01H', 'A01J', 'A01K', 'A01M',
       'A01N',
       ...
       'Y02B', 'Y02C', 'Y02D', 'Y02E', 'Y02P', 'Y02T', 'Y02W', 'Y04S', 'Y10S',
       'Y10T'],
      dtype='object', length=633)

In [13]:
# hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS =1
LEARNING_RATE = 1e-05

In [14]:

from transformers import BertTokenizer, BertModel

In [15]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [16]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['text']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }


In [17]:
train_size = 0.8
train_df = patents1.sample(frac=train_size, random_state=200).reset_index(drop=True)
# val_df = train_df.drop(patents1.index).reset_index(drop=True)
val_size=0.2
val_df = patents1.sample(frac=val_size, random_state=200).reset_index(drop=True)


In [18]:
train_df.shape

(80, 634)

In [19]:
val_df.shape

(20, 634)

In [20]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

In [21]:
train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [22]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [23]:
device

device(type='cuda')

In [24]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [25]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 633)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [26]:

def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [27]:
val_targets=[]
val_outputs=[]

In [40]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf


  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
      """
        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss
      """
    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model


In [41]:
checkpoint_path = "/content/drive/MyDrive/checkpoint_path"
best_model_path = "/content/drive/MyDrive/best_model_path"

In [42]:
import os

ckpt_path=os.makedirs("/content/drive/MyDrive/checkpoint_path", exist_ok=True)

best_model_path=os.makedirs("/content/drive/MyDrive/best_model_path", exist_ok=True)



In [43]:
print("Training DataLoader Length:", len(train_data_loader))
print("Validation DataLoader Length:", len(val_data_loader))


Training DataLoader Length: 3
Validation DataLoader Length: 1


In [44]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer,checkpoint_path, best_model_path)
trained_model

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.233265 	Average Validation Loss: 0.685631
############# Epoch 1  Done   #############



BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [45]:
example=patents1["text"][0]
example

'1. A production method of a hydrogen production filter using a thin film support substrate, comprising a film forming step of forming a Pd alloy film by plating on one surface of a metal base member that is capable of selective etching relative to the thin film support substrate comprising a metal substrate, a plurality of columnar convex portions formed on one surface of said metal substrate, and a plurality of through holes formed at a portion where said columnar convex portions are not formed, so as to pierce said metal substrate, wherein an area of the columnar convex portion non-formed portion occupying on the columnar convex portion formed side is within the range of 20 to 90%, a diffusion joining step of disposing said metal base member on a surface of said thin film support substrate where the columnar convex portions are formed, by diffusion joining said Pd alloy film to the top surfaces of the columnar convex portions, and a removal step of removing said metal base member by

In [46]:
# testing
example = train_df['text'][2]
encodings =tokenizer.encode_plus(
    example,
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)
model.eval()
with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])


B29C


In [47]:
example = train_df['text'][2]
example

'1. A method for communication between radio stations in a radio communication system using a frequency band divided into subcarriers, comprising: transmitting messages between a first radio station and a second radio station by radio in a plurality of hops respectively between adjacent radio stations of adjacent hierarchical levels using one of a plurality of subsets of the subcarriers for each hop and varying a composition of at least one of the subsets used over time, the first radio station being associated with a highest hierarchical level and the radio stations of each lower hierarchical level able to communicate directly with another radio station of a next higher hierarchical level but not with any of the radio stations in any hierarchical level higher than the next higher hierarchical level; allocating by the first radio station to second radio stations of a second hierarchical level a first subset of the subcarriers allocated thereto for communication with third radio station

In [ ]:
patents[patents["text"]=="1. A semiconductor device comprising: a first capacitor; a second capacitor; a third capacitor; a first transistor; and a second transistor, wherein one electrode of each of the first to third capacitors is electrically connected to a gate of the second transistor, wherein the gate of the second transistor is electrically connected to one of a source and a drain of the first transistor, and wherein a channel formation region of the first transistor comprises an oxide semiconductor."]
